In [5]:
import time, os, json, os, math
from collections import Counter,OrderedDict
from typing import Any, Optional
from pprint import pprint

import pandas as pd
import numpy as np

import fuzzywuzzy as fw
from fuzzywuzzy import fuzz

import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')

import torch
import torchtext as tt
from torchtext.data.metrics import bleu_score

from tqdm.notebook import tqdm, trange
device = 'cuda' if torch.cuda.is_available() else 'cpu'

[nltk_data] Downloading package wordnet to /home/ec3dev/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
print(device)

cuda


In [6]:
def get_table_names(path:str = "../../data/en/tables",
                is_subset: bool = False,
                subset_path:Optional[str] = "../input/xinfotabs/subsets/subset_300.json") -> Any:

    infotabs_tables = os.listdir(path)
    if is_subset:
        with open(subset_path, "r") as file:
            subset = json.loads(file.read())
        return (infotabs_tables, subset)
    else: 
        return (infotabs_tables, None)

In [11]:
# --- TEST ONLY; DO NOT INCLUDE ---
all_tables = get_table_names("../data/en/tables")
print(len(all_tables))

2719


In [12]:
# --- TEST ONLY; DO NOT INCLUDE ---
all_tables, subset = get_table_names("../data/en/tables", True, "../utilities/additional_data/subsets/subset_300.json")
print(f"{len(all_tables)} {len(subset['subset'])}")

2719 303


In [13]:
def get_context_table (path: str = "../../utilities/additional_data/table_categories.tsv"):
    context_table = pd.read_csv(path, delimiter='\t', index_col=0)
    return context_table

In [14]:
# --- TEST ONLY; DO NOT INCLUDE ---
cnt_table = get_context_table('../utilities/additional_data/table_categories.tsv')
cnt_table.head()

,category
table_id,
T187,WineYard
T639,WineYard
T1008,WineYard
T1328,WineYard
T1003,Sports Event


In [15]:
def json_to_dict(path : str) -> dict:
    with open(path, 'r') as file:
        result_dict = dict(json.loads(file.read()))
    return result_dict

In [ ]:
def get_linearized_tables(tables_path:str,
                          context_table_path:str,
                          is_subset:bool=False,
                          subset_path:str):
    table_names, subset = get_table_names(tables_path, is_subset, subset_path)
    
    
    for table_code in subset['subset']:
        # loading the table
        table_path = infotabs_tables_path + '/' + table_code + '.json'
        table = json_to_dict(table_path)

        # getting the category
        try:
            category = cat_table.loc[table_code].category
        except:
            category = 'None'

        # getting all the values for each key  
        for key in table.keys():
            subset_texts += [category + ' | ' + key + ' | ' + value for value in table[key]]

    subset_texts = list(set(subset_texts))
    print(f"Length of the subset inputs: {len(subset_texts)}")